# SHAP를 이용한 모델 예측 결과 해석

SHAP 라이브러리를 사용하여 모델이 **'왜' 특정 플레이어를 치터로 판단했는지** 그 근거를 시각적으로 설명

---

### 주요 동작 흐름

1.  **모델 및 데이터 로드**:
    * 학습한 XGBoost 모델을 로드
    * 분석할 대상의 데이터를 로드
2.  **SHAP 값 계산**:
    * 불러온 모델과 데이터를 사용하여 SHAP Explainer를 생성
    * Explainer를 통해 각 특징이 모델 예측에 미친 영향력을 계산
3.  **결과 분석 및 시각화**:
    * 단일 예측에 대해 각 특징이 긍정적(치터일 확률을 높임) 또는 부정적(사람일 확률을 높임)으로 얼마나 기여했는지 시각화
    * 전체 데이터셋에 걸쳐 어떤 특징이 전역적으로 가장 중요한 영향력을 가졌는지 시각화

---

### 핵심 포인트

* **모델 투명성 확보**: 모델이 단순히 '치터다/아니다'라는 결과만 보여주는 것을 넘어, 어떤 특성이 그러한 판단의 결정적인 근거가 되었는지 설명
* **시각적 근거 제시**: SHAP 플롯을 통해 모델의 복잡한 내부 동작을 직관적으로 이해할 수 있는 시각 자료를 생성

## 라이브러리 설치

In [ ]:
!pip install shap

In [ ]:
import xgboost as xgb
import pandas as pd
import numpy as np
import shap
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

# JavaScript 시각화를 위해 초기화 (Colab에서 필수)
shap.initjs()

## 모델 및 데이터 로드

In [ ]:
# --- 불러올 모델의 경로 지정 ---
model_load_path = '/content/drive/MyDrive/Preprocessed_Dataset/xgboost_cheater_detection_model.json'

# --- 모델 로드 ---
model = xgb.XGBClassifier()
model.load_model(model_load_path)

In [ ]:
# --- 불러올 데이터의 경로 지정 ---
data_load_path = '/content/drive/MyDrive/Preprocessed_Dataset/001_data.xlsx'

# --- 데이터 로드 ---
df = pd.read_excel(data_load_path)

In [ ]:
# --- 데이터 전처리 ---

'''
01_training_xgboost_model.ipynb와 동일하게 데이터 전처리를 진행
'''

X_test_df = df.drop(columns=['cheater'])
y_test_df = df['cheater']

## SHAP 값 계산

In [ ]:
# --- SHAP Explainer 생성 ---
explainer = shap.TreeExplainer(model)

In [ ]:
# --- SHAP 값 계산 ---
shap_values = explainer.shap_values(X_test_df)

# --- SHAP Interaction 값 계산 ---
shap_interaction_values = explainer.shap_interaction_values(X_test_df)

## 결과 분석 및 시각화

In [ ]:
# --- 특정 영상 분석 ---
shap.plots.waterfall(shap_values[0])
shap.plots.force(shap_values[0])
shap.plots.bar(shap_values[0])

In [ ]:
# --- 모델 전체 경향성 분석 ---
shap.summary_plot(shap_values[1], X_test_df)
shap.summary_plot(shap_values[1], X_test_df, plot_type="bar")
shap.summary_plot(shap_interaction_values, X_test_df)